In [1]:
import os
import shutil
import glob
import time
from tqdm import tqdm
import random
import numpy as np
import xarray as xr
import pandas as pd
import tensorflow as tf

# Build up efficient TF data pipelines from netCDF-files

We test two different approaches to build up the dataset input streams. <br>
The first one is based on `open_dataset` and requires a large buffer size to enable proper sampling (buffer size $\mathcal{O}(10^4)$ so that at minimum 10 files are buffered). This is due to the fact that only the data files are randomized, not the data samples itself as in the next approach. <br>
The second approach is based on `open_mfdatset` which makes the data sampling much easier since it allows randomization on an index-list for the time dimension to build up the iterator.

In [2]:
def load_nc_dir_with_generator(dir_, patt, shuffle=True, seed=42):
    """
    Opens datafiles via looping in the generator-method. This implies a larger buffer_size (> 12*744 where 12 corresponds to months per year
    and 744 is the number of time steps in a monthly datafile) when shuffling since the buffer gets filled up with data from sequential (unordered) files.
    :param dir_: The directory where the netCDF-files are located.
    :param patt: Substring-pattern to identify the desired netCDF-files ("{patt}*.nc" is applied for searching)
    :param shuffle: flag to enable shuffling
    :param seed: seed for random shuffling
    :return: tf.Dataset for data streain in neural networks
    """
    
    nc_files = glob.glob(os.path.join(dir_, f"{patt}*.nc"))

    if shuffle:
        random.seed(seed)
        random.shuffle(nc_files)
    
    def gen(nc_files, shuffle=True, seed=42):

        for file in nc_files:
            ds = xr.open_dataset(file, engine='netcdf4')
            ntimes = len(ds["time"])
            for t in range(ntimes):
                ds_t = ds.isel({"time": t})
                data_dict = {key: tf.convert_to_tensor(val) for key, val in ds_t.items()}
                data_dict["time"]= np.array([pd.to_datetime(ds_t["time"].values).strftime("%Y-%m-%d %H:%M")])
                yield data_dict


    sample = next(iter(gen(nc_files, shuffle, seed=seed)))
    
    gen_mod = gen(nc_files, shuffle, seed)

    return tf.data.Dataset.from_generator(
        lambda: gen_mod,
        output_signature={
            key: tf.TensorSpec(val.shape, dtype=val.dtype)
            for key, val in sample.items()
        }
    )

def load_mfnc_dir_with_generator(dir_: str, patt: str, shuffle: bool = True, seed: int = 42):
    """
    Opens netCDF-files using xarray's open_mfdataset-method. Shuffling of the data is achieved by shuffling over the time step-indices.
    For efficiency, decoding of the time is disabled (implying shared time-units for all netCDF-data to avoid overwriting with open_mfdataset!!!) 
    since this information is not required for data streaming.
    :param dir_: The directory where the netCDF-files are located.
    :param patt: Substring-pattern to identify the desired netCDF-files ("{patt}*.nc" is applied for searching)
    :param shuffle: flag to enable shuffling
    :param seed: seed for random shuffling
    :return: tf.Dataset for data streain in neural networks
    """    
    ds_all = xr.open_mfdataset(os.path.join(dir_, f"{patt}*.nc"), cache=False, decode_cf=False)
    ntimes = len(ds_all["time"])
    if shuffle: 
        random.seed(seed)
        time_list = random.sample(range(ntimes), ntimes)
    else:
        time_list = range(ntimes)   


    def gen(ds_all):
        #ds_all = xr.open_mfdataset(os.path.join(dir_, f"{patt}*.nc"), cache=False, decode_cf=False)#, parallel=True)#, decode_times=False)       
        for t in time_list:
            # ds = xr.decode_cf(ds_all.isel({"time": t}))
            ds = ds_all.isel({"time": t})
            data_dict = {key: tf.convert_to_tensor(val) for key, val in ds.items()}
            # data_dict["time"]= np.array([pd.to_datetime(ds["time"].values).strftime("%Y-%m-%d %H:%M")])
            yield data_dict        

                    
    sample = next(iter(gen(ds_all)))
    
    gen_mod = gen(ds_all)
    
    return tf.data.Dataset.from_generator(
        lambda: gen_mod,
        output_signature={
            key: tf.TensorSpec(val.shape, dtype=val.dtype)
            for key, val in sample.items()
        }
    )


### highly in efficient in terms of memory -> not tested subsequently!!!
def load_data(dir_, patt) -> xr.DataArray:
    """
    Obtain the data and meta information from the netcdf files, including the len of the samples, mim and max values
    return: data as xarray's DataArray with dimensions [channels, time, lat, lon]
    """

    def reshape_ds(ds):
        da = ds.to_array(dim="variables")
        da = da.transpose(..., "variables")
        return da
    
    ds = xr.open_mfdataset(os.path.join(dir_, f"{patt}*.nc"), cache=False, parallel=True)
    da = reshape_ds(ds)
    init_times = da["time"]
    
    nvars = len(da["variables"])

    return da, init_times #da.chunk(chunks={"time": 744, "variables": nvars}), init_times

After setting up the data directory, both strategies are benchmarked.

In [3]:
datadir = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap5_michael/preprocessed_era5_ifs/netcdf_data/all_files/"
pattern = "preproc_"

We create the respective TF datasets, ...

In [4]:
tfds_test = load_nc_dir_with_generator(datadir, pattern)
tfds_test_mf = load_mfnc_dir_with_generator(datadir, pattern)

2022-07-13 16:25:16.528116: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-13 16:25:17.617259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31019 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


... configure them and... 

In [5]:
sleep_sec = 0
ap1 = iter(tfds_test.shuffle(buffer_size=20000).batch(32).prefetch(100).repeat(1))
ap2 = iter(tfds_test_mf.batch(32).prefetch(100).repeat(1))

2022-07-13 16:25:50.144503: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


then run both approaches. We start with approach 1:

In [6]:
%%time

sleep_sec = 0

for i in tqdm(range(10)):
    if i == 1:
        time_s = time.time()
    
    batch = ap1.get_next()
    print(batch["2t_in"])
    print("***************")
   
load_time_ap1 = (time.time() - time_s)/9.
print("After filling the buffer, retrieving each minibatch took: {0:5.04f}s".format(load_time_ap1))

  0%|          | 0/10 [00:00<?, ?it/s]2022-07-13 16:26:00.209353: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 1258 of 20000
2022-07-13 16:26:10.208179: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 2446 of 20000
2022-07-13 16:26:20.210874: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 3646 of 20000
2022-07-13 16:26:30.217325: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 4877 of 20000
2022-07-13 16:26:40.208714: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 6083 of 20000
2022-07-13 16:26:50.207312: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 7261 of 20000
2022-07-13 16:27:00.210052: I tensorflow/core/kernels/da

tf.Tensor(
[[[284.54684448 284.54766846 284.5475769  ... 280.60705566 280.50146484
   280.39590454]
  [284.56790161 284.57220459 284.57519531 ... 280.45059204 280.35202026
   280.25344849]
  [284.58990479 284.60031128 284.61026001 ... 280.29412842 280.20257568
   280.11102295]
  ...
  [281.30834961 281.26651001 281.22348022 ... 281.23043823 281.17019653
   281.10198975]
  [281.35726929 281.33880615 281.32748413 ... 281.21542358 281.16644287
   281.11279297]
  [281.40475464 281.07073975 281.0769043  ... 281.20953369 281.16818237
   281.12527466]]

 [[287.03387451 287.03518677 287.03536987 ... 283.34674072 283.29367065
   283.2406311 ]
  [287.06460571 287.07086182 287.07546997 ... 283.27249146 283.22360229
   283.17468262]
  [287.09594727 287.11038208 287.1242981  ... 283.19824219 283.15350342
   283.10876465]
  ...
  [282.04986572 281.9960022  281.94046021 ... 285.07333374 285.09609985
   285.11639404]
  [282.11691284 282.09432983 282.08148193 ... 285.12072754 285.1267395
   285.1351623

 20%|██        | 2/10 [02:46<09:08, 68.62s/it] 

tf.Tensor(
[[[287.925354   287.92514038 287.92666626 ... 288.7467041  288.69491577
   288.64312744]
  [287.90405273 287.89959717 287.89755249 ... 288.728302   288.68939209
   288.65048218]
  [287.87857056 287.86514282 287.85247803 ... 288.7098999  288.68386841
   288.65783691]
  ...
  [287.05245972 286.95916748 286.86392212 ... 290.32952881 290.38696289
   290.45962524]
  [287.13687134 287.0881958  287.05221558 ... 290.34347534 290.39141846
   290.44818115]
  [287.22085571 286.45797729 286.45947266 ... 290.34759521 290.38754272
   290.43161011]]

 [[287.15948486 287.16278076 287.16717529 ... 291.25668335 291.26333618
   291.26998901]
  [287.19525146 287.20220947 287.2097168  ... 291.34783936 291.35256958
   291.3572998 ]
  [287.22354126 287.23330688 287.24325562 ... 291.43899536 291.44180298
   291.4446106 ]
  ...
  [291.12841797 291.09762573 291.06564331 ... 292.61312866 292.6690979
   292.74795532]
  [291.17337036 291.16165161 291.15606689 ... 292.60638428 292.66000366
   292.7250061

 30%|███       | 3/10 [02:46<04:21, 37.41s/it]

...

tf.Tensor(
[[[290.04193115 290.04382324 290.04644775 ... 287.88464355 287.7991333
   287.71359253]
  [290.06079102 290.06445312 290.0687561  ... 287.67648315 287.6055603
   287.53466797]
  [290.07476807 290.07897949 290.08340454 ... 287.46829224 287.41201782
   287.35571289]
  ...
  [291.63891602 291.53433228 291.42843628 ... 290.01620483 290.0713501
   290.13735962]
  [291.70489502 291.64224243 291.590271   ... 290.04580688 290.08673096
   290.13543701]
  [291.77346802 290.66452026 290.65280151 ... 290.06195068 290.09362793
   290.12957764]]

 [[279.45684814 279.45687866 279.45675659 ... 277.75668335 277.7612915
   277.76586914]
  [279.45895386 279.4593811  279.45944214 ... 277.72589111 277.72668457
   277.72747803]
  [279.46166992 279.46328735 279.4647522  ... 277.69509888 277.69207764
   277.68908691]
  ...
  [277.80197144 277.730896   277.65835571 ... 280.77575684 280.79281616
   280.81542969]
  [277.86447144 277.82684326 277.79867554 ... 280.753479   280.77526855
   280.79785156]


 90%|█████████ | 9/10 [02:48<00:03,  3.26s/it]

tf.Tensor(
[[[275.47036743 275.46685791 275.46057129 ... 269.35702515 269.25683594
   269.15661621]
  [275.45010376 275.44641113 275.44012451 ... 269.25241089 269.15512085
   269.05783081]
  [275.44158936 275.44262695 275.44281006 ... 269.14776611 269.05340576
   268.95904541]
  ...
  [278.71685791 278.6991272  278.68057251 ... 273.62014771 273.6020813
   273.57839966]
  [278.7460022  278.73983765 278.73745728 ... 273.63803101 273.61660767
   273.59396362]
  [278.77386475 277.93341064 277.94750977 ... 273.65283203 273.63128662
   273.61044312]]

 [[276.74459839 276.73690796 276.72607422 ... 266.22189331 266.20681763
   266.19171143]
  [276.66793823 276.65307617 276.63601685 ... 265.82052612 265.82449341
   265.82843018]
  [276.6104126  276.59240723 276.57354736 ... 265.41912842 265.44216919
   265.46517944]
  ...
  [280.01739502 280.05279541 280.0892334  ... 271.95114136 271.91094971
   271.8626709 ]
  [279.97528076 279.99151611 280.00195312 ... 271.93634033 271.90460205
   271.8676757

100%|██████████| 10/10 [02:48<00:00, 16.87s/it]

tf.Tensor(
[[[292.04806519 292.05078125 292.05584717 ... 294.21487427 294.20236206
   294.18988037]
  [292.06195068 292.06442261 292.0690918  ... 294.37701416 294.35693359
   294.33685303]
  [292.06637573 292.06448364 292.06332397 ... 294.53915405 294.51147461
   294.48382568]
  ...
  [296.40579224 296.32788086 296.2484436  ... 297.40942383 297.47381592
   297.55966187]
  [296.47311401 296.43170166 296.40054321 ... 297.39279175 297.45623779
   297.5291748 ]
  [296.54037476 295.96981812 295.96859741 ... 297.37362671 297.43151855
   297.49227905]]

 [[289.64172363 289.64471436 289.64804077 ... 291.16470337 291.14056396
   291.11645508]
  [289.68075562 289.68844604 289.69598389 ... 291.23364258 291.21502686
   291.19641113]
  [289.71435547 289.72729492 289.74020386 ... 291.30258179 291.28948975
   291.27639771]
  ...
  [294.88909912 294.80682373 294.72314453 ... 292.03414917 292.11752319
   292.21383667]
  [294.95294189 294.90750122 294.87213135 ... 292.06729126 292.13192749
   292.205230

Now, we continue with approach 2:

In [7]:
%%time

time_s = time.time()

for i in tqdm(range(10)):
    if i == 0:
        print(f"Sleeping for {sleep_sec}s...")
        time.sleep(sleep_sec)
    
    batch = ap2.get_next()
    print(batch["2t_in"])
    print("***************")
    
load_time_ap2 = (time.time() - time_s)/10.


  0%|          | 0/10 [00:00<?, ?it/s]

Sleeping for 0s...


 10%|█         | 1/10 [00:02<00:18,  2.05s/it]

tf.Tensor(
[[[285.24121094 285.24240112 285.24356079 ... 283.93896484 283.92404175
   283.90908813]
  [285.25860596 285.26208496 285.2651062  ... 283.84768677 283.8348999
   283.82211304]
  [285.27450562 285.28115845 285.28765869 ... 283.75643921 283.74578857
   283.73516846]
  ...
  [282.3638916  282.27709961 282.18832397 ... 281.96502686 282.0050354
   282.03707886]
  [282.447052   282.4029541  282.37115479 ... 282.01016235 282.03088379
   282.05023193]
  [282.52938843 281.88253784 281.88616943 ... 282.04141235 282.05410767
   282.0668335 ]]

 [[285.88839722 285.88092041 285.8706665  ... 281.9781189  281.93240356
   281.88668823]
  [285.81097412 285.79589844 285.77868652 ... 281.85232544 281.80548096
   281.75863647]
  [285.7522583  285.73376465 285.71444702 ... 281.72653198 281.67858887
   281.63061523]
  ...
  [288.64047241 288.70535278 288.77069092 ... 283.64855957 283.69055176
   283.73916626]
  [288.61199951 288.65396118 288.69070435 ... 283.66793823 283.69976807
   283.73623657

 20%|██        | 2/10 [00:05<00:20,  2.59s/it]

...

tf.Tensor(
[[[284.35925293 284.35949707 284.35772705 ... 282.21395874 282.20574951
   282.19754028]
  [284.38375854 284.3888855  284.39144897 ... 282.3104248  282.30697632
   282.30355835]
  [284.41271973 284.42773438 284.44192505 ... 282.40689087 282.40823364
   282.40957642]
  ...
  [283.94430542 283.86236572 283.77868652 ... 291.10678101 291.20761108
   291.35903931]
  [284.01968384 283.97738647 283.94644165 ... 291.01889038 291.13806152
   291.27630615]
  [284.09451294 283.4493103  283.4508667  ... 290.94174194 291.05807495
   291.1784668 ]]

 [[286.53646851 286.53735352 286.5385437  ... 287.60992432 287.51412964
   287.41830444]
  [286.54598999 286.54782104 286.54971313 ... 287.70922852 287.60897827
   287.50872803]
  [286.55374146 286.55661011 286.55947876 ... 287.8085022  287.7038269
   287.59912109]
  ...
  [290.06622314 290.07684326 290.08703613 ... 291.22424316 291.24169922
   291.27423096]
  [290.07763672 290.08889771 290.10128784 ... 291.1652832  291.19909668
   291.2358703

 90%|█████████ | 9/10 [00:19<00:02,  2.16s/it]

tf.Tensor(
[[[286.57025146 286.57321167 286.57684326 ... 288.03390503 288.01611328
   287.99832153]
  [286.60568237 286.6126709  286.62008667 ... 288.09182739 288.06686401
   288.04187012]
  [286.63452148 286.64456177 286.65484619 ... 288.14978027 288.11761475
   288.0854187 ]
  ...
  [288.72567749 288.67001343 288.61297607 ... 291.40179443 291.47573853
   291.55838013]
  [288.78326416 288.75616455 288.73748779 ... 291.4473877  291.49975586
   291.5597229 ]
  [288.83984375 288.35351562 288.35873413 ... 291.47427368 291.5140686
   291.55819702]]

 [[280.84951782 280.84899902 280.84457397 ... 275.57458496 275.50695801
   275.43936157]
  [280.88442993 280.8918457  280.89413452 ... 275.33566284 275.27865601
   275.22164917]
  [280.93011475 280.95562744 280.97940063 ... 275.09671021 275.05032349
   275.00390625]
  ...
  [274.03015137 273.94070435 273.84970093 ... 272.20092773 272.18276978
   272.16677856]
  [274.10101318 274.0519104  274.01385498 ... 272.20895386 272.19137573
   272.1761474

100%|██████████| 10/10 [00:21<00:00,  2.19s/it]

tf.Tensor(
[[[277.31997681 277.31295776 277.30096436 ... 275.68490601 275.65612793
   275.62734985]
  [277.2729187  277.26419067 277.25100708 ... 275.68664551 275.65231323
   275.61798096]
  [277.24807739 277.24597168 277.24240112 ... 275.68835449 275.64849854
   275.60861206]
  ...
  [275.89181519 275.84567261 275.79873657 ... 275.07290649 275.1362915
   275.20870972]
  [275.92840576 275.90304565 275.8833313  ... 275.10360718 275.15109253
   275.2052002 ]
  [275.96533203 275.5475769  275.54562378 ... 275.12042236 275.1579895
   275.19891357]]

 [[279.41421509 279.41439819 279.41479492 ... 278.37481689 278.28005981
   278.18530273]
  [279.41418457 279.41415405 279.41436768 ... 278.565979   278.47189331
   278.3777771 ]
  [279.41333008 279.41259766 279.41195679 ... 278.75714111 278.66372681
   278.57028198]
  ...
  [281.19512939 281.17840576 281.16122437 ... 285.98464966 286.05633545
   286.15655518]
  [281.21481323 281.20755005 281.20318604 ... 285.92266846 286.00613403
   286.09893799

### Results

In [9]:
print("After filling the buffer, retrieving each minibatch with APPROACH 1 took: {0:5.04f}s".format(load_time_ap1))
print("Retrieving each minibatch with APPROACH 2 took: {0:5.04f}s".format(load_time_ap2))

After filling the buffer, retrieving each minibatch with APPROACH 1 took: 0.2679s
Retrieving each minibatch with APPROACH 2 took: 2.1932s


Thus, we see that the first approach outperforms the second approach (at least after the buffer has been filled once).